In [1]:
import torch
import torch_geometric.transforms as T
import seaborn as sns
import matplotlib.pyplot as plt
from torch_geometric.loader.link_neighbor_loader import LinkNeighborLoader
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
import torch.functional as F
import torch
import pickle 
#from gat_dependency.GAT_model import HeteroData_GNNmodel
from gat_dependency.HetGNN_Model_Jovana import HeteroData_GNNmodel
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import os
import argparse
from datetime import datetime
from torch_geometric.explain import Explainer, GNNExplainer, ModelConfig
from torch_geometric import seed_everything
from copy import deepcopy

In [2]:
heterodata_obj = torch.load('Data/multigraphs/heteroData_gene_cell_Neuroblastoma_Reactome_crispr-1_5_cgp_cnv.pt')
heterodata_obj

HeteroData(
  gene={
    node_id=[14034],
    names=[14034],
    x=[14034, 3438],
  },
  cell={
    node_id=[37],
    names=[37],
    x=[37, 1218],
  },
  (gene, interacts_with, gene)={ edge_index=[2, 278974] },
  (gene, dependency_of, cell)={ edge_index=[2, 14099] },
  (gene, rev_interacts_with, gene)={ edge_index=[2, 278974] },
  (cell, rev_dependency_of, gene)={ edge_index=[2, 14099] }
)

In [3]:
metapaths = [[('gene', 'dependency_of', 'cell'), ('cell', 'rev_dependency_of', 'gene')],
             [('cell', 'rev_dependency_of', 'gene'), ('gene', 'dependency_of', 'cell')],
             #[('gene', 'interacts with', 'gene')]
             ]

transform = T.AddMetaPaths(
    metapaths=metapaths, 
    keep_same_node_type= True,
    drop_unconnected_node_types=False, 
    weighted=True,
    #drop_orig_edge_types=True
)
heterodata_obj = transform(heterodata_obj)  
heterodata_obj

/opt/homebrew/lib/python3.11/site-packages/torch_geometric/edge_index.py:2007: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:55.)
  out = torch.matmul(sparse_input, other)


HeteroData(
  metapath_dict={
    (gene, metapath_0, gene)=[2],
    (cell, metapath_1, cell)=[2],
  },
  gene={
    node_id=[14034],
    names=[14034],
    x=[14034, 3438],
  },
  cell={
    node_id=[37],
    names=[37],
    x=[37, 1218],
  },
  (gene, interacts_with, gene)={ edge_index=[2, 278974] },
  (gene, dependency_of, cell)={ edge_index=[2, 14099] },
  (gene, rev_interacts_with, gene)={ edge_index=[2, 278974] },
  (cell, rev_dependency_of, gene)={ edge_index=[2, 14099] },
  (gene, metapath_0, gene)={
    edge_index=[2, 721140],
    edge_weight=[721140],
  },
  (cell, metapath_1, cell)={
    edge_index=[2, 1369],
    edge_weight=[1369],
  }
)

In [4]:
def count_disconnected_genes(hetero_data):
    # Get the edge index for the gene-cell connection
    gene_to_cell_edge_index = hetero_data['gene', 'dependency_of', 'cell'].edge_index

    # Get the set of all gene nodes
    total_genes = hetero_data['gene'].num_nodes
    all_gene_nodes = torch.arange(total_genes)

    # Find unique gene nodes that are connected to any cell
    connected_gene_nodes = torch.unique(gene_to_cell_edge_index[0])

    # Create a boolean mask for genes that are disconnected
    is_disconnected = ~torch.isin(all_gene_nodes, connected_gene_nodes)

    # Count disconnected genes
    num_disconnected_genes = is_disconnected.sum().item()
    num_connected_genes = connected_gene_nodes.size()
    return num_disconnected_genes, num_connected_genes



x = count_disconnected_genes(heterodata_obj)
x

(13034, torch.Size([1000]))

In [5]:
cell_metapath_edge_index = heterodata_obj['cell', 'metapath_1', 'cell'].edge_index

# Get all unique nodes involved in metapath_1
unique_cells_in_metapath_1 = torch.unique(cell_metapath_edge_index)

# Count the number of unique cell nodes in metapath_1
num_unique_cells_in_metapath_1 = unique_cells_in_metapath_1.size(0)

print(f"Number of unique cell nodes in metapath_1: {num_unique_cells_in_metapath_1}")


Number of unique cell nodes in metapath_1: 37


In [6]:
x = 0
for i in range(37):
    x = x +i

x*2 + 37

1369

In [7]:
def construct_complete_predMatrix(total_predictions: np.array,
                                  edge_index: torch.Tensor,
                                  index: list, columns: list) -> pd.DataFrame:
    total_preds_df = pd.DataFrame({"gene":edge_index[0], "cell":edge_index[1], "prob": total_predictions})
    # total_preds_df['gene'] = total_preds_df.gene.apply(lambda x: ppi_int2gene[x])
    # total_preds_df['cell'] = total_preds_df.cell.apply(lambda x: int2cell[x])

    dep_df = pd.DataFrame(index=index, columns=columns, dtype=float) 
    for i in range(dep_df.shape[0]):
        tmp = total_preds_df.iloc[i*dep_df.shape[1]:(i+1)*dep_df.shape[1]]
        dep_df.loc[tmp.cell.iloc[0], tmp.gene.values] = tmp['prob'].values

    return dep_df

In [8]:
# if torch.backends.mps.is_available():
#    device = torch.device("mps")
#    x = torch.ones(1, device=device)
#    print (x)
# else:
#    print ("MPS device not found.")

device = 'cpu'

In [9]:
cells, genes = heterodata_obj['cell'].names, heterodata_obj['gene'].names

cell2int = dict(zip(heterodata_obj['cell'].names, heterodata_obj['cell'].node_id.numpy()))
gene2int = dict(zip(heterodata_obj['gene'].names, heterodata_obj['gene'].node_id.numpy()))
dep_genes = list(set(heterodata_obj['gene', 'dependency_of', 'cell'].edge_index[0].numpy())) # all genes that have a dependency edge

crispr_neurobl = pd.read_csv("./Data/crispr_Neuroblastoma_Reactome.csv", index_col=0)
crispr_neurobl_int = crispr_neurobl.copy(deep=True)
crispr_neurobl_int.index = [cell2int[i] for i in crispr_neurobl.index]
crispr_neurobl_int.columns = [gene2int[i] for i in crispr_neurobl.columns]
crispr_neurobl_int = crispr_neurobl_int.loc[:, dep_genes] # only keep the genes that have a dependency edge
crispr_neurobl_bin = crispr_neurobl_int.applymap(lambda x: int(x < -1.5)) # binarize crispr data (-1,5 is threshold for positive dependency)
del heterodata_obj['gene'].names, heterodata_obj['cell'].names

/var/folders/ds/76n_grr91zn97zn1yl541y5c0000gn/T/ipykernel_65242/683908446.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  crispr_neurobl_bin = crispr_neurobl_int.applymap(lambda x: int(x < -1.5)) # binarize crispr data (-1,5 is threshold for positive dependency)


In [10]:
hidden_features = [(-1,-1),256,128] #'How many hidden features for each GNN layer')
heads = [1,1] #'Number of multiheads to use per GATlayer, must be same length as hidden features')??

node_types = ['gene', 'cell']
features_dim = {'gene': heterodata_obj['gene'].x.shape[1],
                'cell': heterodata_obj['cell'].x.shape[1]}

emb_dim = int(512)

In [ ]:
hetGNNmodel = HeteroData_GNNmodel(heterodata=heterodata_obj,   #model defined in GAT_model.py
                                  node_types=node_types,
                                  node_types_to_pred = node_types,
                                  embedding_dim=emb_dim,
                                  #gcn_model=args.gcn_model,
                                  features=hidden_features,
                                  #layer_name=args.layer_name,
                                  heads=heads,
                                  dropout=args.dropout,
                                  act_fn=torch.nn.ReLU,
                                  #lp_model=args.lp_model,
                                  lp_model='simple',
                                  #add_self_loops=False, #DIT MOET JE WEL TOEVOEGEN!!!
                                  features_dim=features_dim,
                                  aggregate='sum',
                                  return_attention_weights=False)

hetGNNmodel.to(device)
print(hetGNNmodel)

HeteroData_GNNmodel(
  (nt1_lin): Linear(in_features=3438, out_features=512, bias=True)
  (nt2_lin): Linear(in_features=1218, out_features=512, bias=True)
  (nt1_emb): Embedding(14034, 512)
  (nt2_emb): Embedding(37, 512)
  (conv1): HeteroConv(num_relations=4)
  (conv2): HeteroConv(num_relations=4)
  (act_fn): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (classifier): LPsimple_classif()
)


In [12]:
# Define the full probability matrix for validation
cls_int = heterodata_obj['cell'].node_id
cl_probs = torch.zeros((2, len(cls_int)*len(dep_genes)), dtype=torch.long)
    
for i, cl in enumerate(cls_int):
    # cl = 20
    x_ = torch.stack((torch.tensor(dep_genes), 
                    torch.tensor([cl]*len(dep_genes))), dim=0)
                        
    cl_probs[:, i*len(dep_genes):(i+1)*len(dep_genes)] = x_
full_pred_data = heterodata_obj.clone()
full_pred_data['gene', 'dependency_of', 'cell'].edge_label_index = cl_probs


In [13]:
# Define training parameters
optimizer = torch.optim.Adam(hetGNNmodel.parameters(), lr=0.01)

loss_fn = torch.nn.BCEWithLogitsLoss() # Binary cross entropy loss
best_loss = np.inf
epoch_since_best = 0
best_ap = 0
best_ap_model = None
lowest_loss = np.inf
best_loss_model = None
n_epochs = 3

In [14]:
 # Split graph in train/validation
transform_traintest = T.RandomLinkSplit(num_val=0.1,
                                            num_test=0.2,
                                            disjoint_train_ratio=0,
                                            neg_sampling_ratio=3,
                                            add_negative_train_samples=1,
                                            edge_types=('gene', 'dependency_of', 'cell'),
                                            rev_edge_types=('cell', 'rev_dependency_of', 'gene'),
                                            is_undirected=True)

train_data, val_data, test_data = transform_traintest(heterodata_obj)

    # Define the loaders
train_neg_sampling = True
if train_neg_sampling:
        train_loader = LinkNeighborLoader(data=train_data,
                                        num_neighbors={et: [-1]*2 for et in heterodata_obj.edge_types}, 
                                        edge_label_index=(("gene", "dependency_of", "cell"),
                                                            train_data["gene", "dependency_of", "cell"].edge_label_index),
                                            edge_label=train_data["gene", "dependency_of", "cell"].edge_label,
                                            batch_size=128, # how many pos per batch -> actual batch_size is (npr+1)*batch_size
                                            directed=True, # undirected het graphs not yet supported -> that is why the reverse type is added
                                            shuffle=True,
                                            num_workers=10)
else:
        train_loader = LinkNeighborLoader(data=train_data,
                                        num_neighbors={et: [-1]*2 for et in heterodata_obj.edge_types}, 
                                        neg_sampling_ratio=3,
                                        edge_label_index=(("gene", "dependency_of", "cell"),
                                                            train_data["gene", "dependency_of", "cell"].edge_label_index),
                                            edge_label=train_data["gene", "dependency_of", "cell"].edge_label,
                                            batch_size=128, # how many pos per batch -> actual batch_size is (npr+1)*batch_size
                                            directed=True, # undirected het graphs not yet supported -> that is why the reverse type is added
                                            shuffle=True,
                                            num_workers=10)


In [15]:
val_ratio = 0.1

# Train the model but first delete the names 
assay_ap_total, gene_ap_total = [], []
for epoch in range(n_epochs):
    total_train_loss = 0
    hetGNNmodel.train()
    
    for sampled_data in train_loader:
        optimizer.zero_grad()
        sampled_data.to(device)
        
       
        out = hetGNNmodel(sampled_data, edge_type_label="gene,dependency_of,cell")
        
        ground_truth = sampled_data["gene", "dependency_of", "cell"].edge_label
        loss = loss_fn(out, ground_truth) 
        total_train_loss += loss
        loss.backward()
        optimizer.step()

    # scheduler.step()
    ap_val, auc_val = 0, 0
    hetGNNmodel.eval()
    with torch.no_grad():
        if val_ratio != 0.0:
            val_data.to(device)
           
            out = hetGNNmodel(val_data, edge_type_label="gene,dependency_of,cell")
            
            pred = torch.sigmoid(out)
            ground_truth = val_data["gene", "dependency_of", "cell"].edge_label
            val_loss = loss_fn(out, ground_truth)

            auc_val = roc_auc_score(ground_truth.cpu(), pred.cpu())
            ap_val = average_precision_score(ground_truth.cpu(), pred.cpu())

            if ap_val > best_ap:
                best_ap = ap_val
                best_ap_model = deepcopy(hetGNNmodel.state_dict())
                final_epoch = epoch

        full_pred_data.to(device)
        total_preds = hetGNNmodel(data=full_pred_data, edge_type_label="gene,dependency_of,cell")
        total_preds_out = torch.sigmoid(total_preds).cpu().numpy()
        tot_pred_deps = construct_complete_predMatrix(total_predictions=total_preds_out,
                                                      edge_index=cl_probs,
                                                      index=cls_int.numpy(),
                                                      columns=dep_genes)

        assay_corr = tot_pred_deps.corrwith(crispr_neurobl_int*-1, method='spearman', axis=1)  # correlation with crispr data, multiplied by -1 to reverse the direction of dependency
        gene_ap, assay_ap = [], []
        
        for i, row in tot_pred_deps.iterrows():  # for each cell line
            assay_ap.append(average_precision_score(y_true=crispr_neurobl_bin.loc[i].values,
                                                    y_score=row.values))
        
        for col in tot_pred_deps.columns:  # for each gene
            gene_ap.append(average_precision_score(y_true=crispr_neurobl_bin[col].values,
                                                    y_score=tot_pred_deps[col].values))            

        assay_ap_total.append(assay_ap)  # for all epochs
        gene_ap_total.append(gene_ap)

    print({
        'epoch': epoch, 
        'train loss': total_train_loss/len(train_loader),
        'val loss': val_loss, 
        'val auc': auc_val, 
        'val ap': ap_val,
        'assay_ap': np.mean(assay_ap), 
        'gene_ap': np.mean(gene_ap),
        'assay_corr_sp': assay_corr.mean() 
    })

{'epoch': 0, 'train loss': tensor(8.7286, grad_fn=<DivBackward0>), 'val loss': tensor(0.0906), 'val auc': np.float64(0.9923499494529994), 'val ap': np.float64(0.9663413429105617), 'assay_ap': np.float64(0.6953192560913383), 'gene_ap': np.float64(0.38105405405405407), 'assay_corr_sp': np.float64(0.6189940424140652)}
{'epoch': 1, 'train loss': tensor(0.0844, grad_fn=<DivBackward0>), 'val loss': tensor(0.0851), 'val auc': np.float64(0.9922789267615012), 'val ap': np.float64(0.9647958231696103), 'assay_ap': np.float64(0.6910976883573021), 'gene_ap': np.float64(0.38105405405405407), 'assay_corr_sp': np.float64(0.6432005676281486)}
{'epoch': 2, 'train loss': tensor(0.0831, grad_fn=<DivBackward0>), 'val loss': tensor(0.0860), 'val auc': np.float64(0.9931337175946378), 'val ap': np.float64(0.9714735242242575), 'assay_ap': np.float64(0.7240525965604684), 'gene_ap': np.float64(0.38105405405405407), 'assay_corr_sp': np.float64(0.6614873358860662)}
